<a href="https://colab.research.google.com/github/hiroto-noguchi/useful_materials_for_research/blob/main/csv_to_wav(jsPsych).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 必要ライブラリのインストール
!pip install pydub

# 2. モジュールのインポート
import io
import base64
import glob
import os
import pandas as pd
from pydub import AudioSegment
from google.colab import files  # Colab 限定

# 3. CSV ファイルアップロード UI
print("CSV ファイルをアップロードしてください（html-audio-response を含む）")
uploaded = files.upload()

# 4. アップロードされた CSV を読み込む
df = None
for fname, content in uploaded.items():
    if fname.lower().endswith('.csv'):
        df = pd.read_csv(io.BytesIO(content))
        print(f"Loaded {fname} ({len(df)} rows)")
if df is None:
    raise RuntimeError("CSV ファイルが見つかりませんでした。")

# 5. 出力ディレクトリの作成
output_dir = '/content/audio_outputs'
os.makedirs(output_dir, exist_ok=True)

# 6. Base64 デコード関数
def convert_snd(base64_str):
    return base64.b64decode(base64_str)

# 7. WebM → WAV 変換
for row in df.itertuples():
    if getattr(row, 'trial_type', '') == 'html-audio-response':
        file_base = f"{row.run_id}_{row.trial_index}"
        webm_path = os.path.join(output_dir, f"{file_base}.webm")
        with open(webm_path, 'wb') as f:
            f.write(convert_snd(row.response))
        wav_path = os.path.join(output_dir, f"{file_base}.wav")
        audio = AudioSegment.from_file(webm_path, format="webm")
        audio.export(wav_path, format="wav")
        print(f"Exported {wav_path}")

# 8. WAV ファイルを自動ダウンロード
wav_files = glob.glob(os.path.join(output_dir, '*.wav'))
for wav in wav_files:
    print(f"Downloading {os.path.basename(wav)}...")
    files.download(wav)
